In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from urllib.parse import urlunparse
from selenium import webdriver
import time

In [2]:
browser = webdriver.Chrome("C:/Users/lsy96/chromedriver.exe")
browser.maximize_window()

filters = 'topsellers'
url = f"https://store.steampowered.com/search/?filter={filters}"
browser.get(url)

In [3]:
prev_height = browser.execute_script("return document.body.scrollHeight")

# 맨 아래로 무한 스크롤 내리기
while True:
    # 스크롤을 화면 가장 아래로
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    
    # 페이지 로딩 대기
    time.sleep(2)
    
    # 현재 문서 높이를 가져와서 저장
    curr_height = browser.execute_script("return document.body.scrollHeight")
    
    if(curr_height == prev_height):
        break
    else:
        prev_height = browser.execute_script("return document.body.scrollHeight")
        
url = browser.page_source

In [5]:
source = BeautifulSoup(url, 'lxml')

In [6]:
#1. title -> ok
title_list = []

titles = source.find_all('span', class_='title')

for title in titles:
    title_list.append(title.text)
    

In [7]:
#2. released date -> ok
date_list = []

dates = source.find_all('div', class_='col search_released responsive_secondrow')

for date in dates:
    date_list.append(date.text)
    

In [8]:
#3. price -> ok
price_list = []
discount_price_list = []

prices = source.find_all('div', {'class': {'col search_price responsive_secondrow', 'col search_price discounted responsive_secondrow'}})
#prices = source.select('.col search_price responsive_secondrow, .col search_price discounted responsive_secondrow')

for price in prices:

    price_list.append(price.text.strip().split('₩')[1])


In [9]:
#4. discount -> ok
discount_list = []

discounts = source.find_all('div', class_='col search_discount responsive_secondrow')

for discount in discounts:
    discount_list.append(discount.text.strip())
    

In [10]:
#5. review -> 길이 다름
review_list = []
review_comment_list = []

reviews = source.select('[data-tooltip-html]')

for review in reviews:
    review_list.append(review['data-tooltip-html'].split('<br>')[0])
    review_comment_list.append(review['data-tooltip-html'].split('<br>')[1])


In [11]:
#6. id -> 길이 다름
id_list = []

ids = source.select('[data-ds-appid]')

for di in ids:
    id_list.append(di['data-ds-appid'])
    

In [12]:
dic = {
    'title': title_list, 
    'date': date_list, 
    'price': price_list, 
    'discount': discount_list, 
    'review': review_list, 
    'review_comment': review_comment_list, 
    'id': id_list
}
total = pd.DataFrame.from_dict(dic, orient='index')
total = total.transpose()

In [25]:
# 혹시 필요할까봐 inplace=True 는 나중에 하기로 기억하기!!
real_total = total.dropna()

In [26]:
# game별 세부 페이지 url 추출

for i in range(len(real_total)):
    review_url = f"https://steamcommunity.com/app/{real_total.loc[i, 'id']}/reviews/?browsefilter=toprated&snr=1_5_100010_"
    browser.get(review_url)
    


['https://store.steampowered.com/app/858820',
 'https://store.steampowered.com/app/821880',
 'https://store.steampowered.com/app/578080',
 'https://store.steampowered.com/app/1238840',
 'https://store.steampowered.com/app/1259420',
 'https://store.steampowered.com/app/1522820',
 'https://store.steampowered.com/app/962130',
 'https://store.steampowered.com/app/1123050',
 'https://store.steampowered.com/app/979690',
 'https://store.steampowered.com/app/1097150',
 'https://store.steampowered.com/app/1426210',
 'https://store.steampowered.com/app/476600',
 'https://store.steampowered.com/app/393080',
 'https://store.steampowered.com/app/',
 'https://store.steampowered.com/app/413150',
 'https://store.steampowered.com/app/535930',
 'https://store.steampowered.com/app/1517290',
 'https://store.steampowered.com/app/892970',
 'https://store.steampowered.com/app/771300',
 'https://store.steampowered.com/app/1127500',
 'https://store.steampowered.com/app/648800',
 'https://store.steampowered.com

In [ ]:
browser = webdriver.Chrome("C:/Users/lsy96/chromedriver.exe")
browser.maximize_window()
review_url = f"https://steamcommunity.com/app/{real_total.loc[0, 'id']}/reviews/?browsefilter=toprated&snr=1_5_100010_"
browser.get(review_url)

prev_height = browser.execute_script("return document.body.scrollHeight")

# 맨 아래로 무한 스크롤 내리기
while True:
    # 스크롤을 화면 가장 아래로
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    
    # 페이지 로딩 대기
    time.sleep(2)
    
    # 현재 문서 높이를 가져와서 저장
    curr_height = browser.execute_script("return document.body.scrollHeight")
    
    if(curr_height == prev_height):
        break
    else:
        prev_height = browser.execute_script("return document.body.scrollHeight")
        
review_url = browser.page_source
source_r = BeautifulSoup(review_url, 'lxml')